## The basics

OCaml is very similar to other languages where basic types are concerned.

In [84]:
1

- : int = 1


`: T = V` means that the expression we evaluated to value `V` and has type `T`.   OCaml uses _type inference_ to automatically figure out the type of an expression.  `int` is the type of an integer.

In [85]:
"Hello world!"

- : string = "Hello world!"


In [86]:
1 + 2

- : int = 3


OCaml also has a strong type system that will let you know when you do something stupid.

In [87]:
1 + "String"

error: compile_error

Let's look at a more complex expression such as a list.

In [88]:
[1; 2; 3]

- : int list = [1; 2; 3]


Our list has type `int list`.  You can think of this as a `list` of `int`s.

What type does the empty list `[]` have?

In [89]:
[]

- : 'a list = []


`'a` is a _type_ variable.  This indicates that `[]` is a list of elements of some type that is yet to be determined.

## Functions

In functional languages, functions are expressions.

In [90]:
fun arg -> 42

- : 'a -> int = <fun>


Let's unpack this.  We got back a value of type `'a -> int`, which is a function that takes an undetermined value as an argument, and returns an integer.

Let's give our function a name!

In [91]:
let myfun = fun arg -> 42

val myfun : 'a -> int = <fun>


In [92]:
myfun

- : 'a -> int = <fun>


Our function is now named `myfun`.

In [93]:
myfun 0

- : int = 42


In [94]:
myfun 0.0

- : int = 42


In [95]:
myfun ["Wow this is crazy"]

- : int = 42


We can also use a slightly shorter syntax when we are defining functions.

In [96]:
let myfun arg = 42

val myfun : 'a -> int = <fun>


In [97]:
let succ n = n + 1

val succ : int -> int = <fun>


Look at that.  Now we have a function that takes an integer argument and returns an integer.

In [98]:
succ 0

- : int = 1


In [99]:
succ 42

- : int = 43


In [100]:
let id a = a

val id : 'a -> 'a = <fun>


We just defined a function that takes an undetermined type and returns the same type!

In [101]:
id 0

- : int = 0


In [102]:
id "You won't return this value"

- : string = "You won't return this value"


In [103]:
id [myfun]

- : ('_weak3 -> int) list = [<fun>]


Let's define a function that takes two arguments.

In [104]:
let add a b = a + b

val add : int -> int -> int = <fun>


- We got back a crazy type `int -> int -> int`
- The trick is to add parentheses like  `int -> (int -> int)`
- If you provide an `int`, you get back a new function that takes an `int`
- This idea is called _currying_

Just like we would expect, we can pass both arguments at once.

In [105]:
add 40 2

- : int = 42


But because of currying, we can pass a single argument and we'll still get back a function!  This is called _partial application_.

In [106]:
add 40

- : int -> int = <fun>


And we can save it and call it as usual.

In [107]:
let add1 = add 1 in
add1 41

- : int = 42


## Fun with Lists

### map

Having first-class functions allows us to write a lot of interesting list processing functions.  Let's start with `List.map`.  OCaml organizes code into modules.  `List.map` simply means the `map` symbol in the `List` module.  In this case, `map` is a function.

`map f [e0; e1; ...; en]` is defined to be `[f e0; f e1; ...; f en]`.  In other words, `map` applies the function `f` to every element of the list.

Let's take a look at the type...

In [108]:
List.map

- : ('a -> 'b) -> 'a list -> 'b list = <fun>


This looks pretty intimidating at first, but let's break it down.  First, there are two types, `'a` and `'b`.  The first argument is a function from `'a` to `'b`.  This is the function `f` that is going to be doing the mapping.

The second argument is of type `'a list`, or a list containing `'a` elements.  And the last argument is a list of `'b` elements.

Notice that the type of the first argument (`f`) makes sense, since it will be consuming elements of the first list, which are of type `'a`, and producing elements of the second list, which are of type `'b`.

Let's try a few examples!

In [109]:
List.map (fun x -> x + 42) [1; 2; 3]

- : int list = [43; 44; 45]


In [110]:
let square x = x*x;;
List.map square [0; 1; 2; 3; 4]

val square : int -> int = <fun>


- : int list = [0; 1; 4; 9; 16]


In [111]:
let l = [0; 1; 2; 3; 4] in
let l2 = List.map square l in
let l3 = List.map square l2 in
l3

- : int list = [0; 1; 16; 81; 256]


### Filter

Another very useful function is `List.filter`.  `filter f l` runs `f` on each element of `l`, and returns a list consisting solely of the elements of `l` that `f` returned true on.

In [112]:
List.filter

- : ('a -> bool) -> 'a list -> 'a list = <fun>


In this case, `f` has type `'a -> bool` because it's deciding yes (true) or no (false) for each element of the list.

Let's try a few examples.

In [113]:
let is_positive x = x > 0 in
List.filter is_positive [-2; -1; 0; 1; 2]

- : int list = [1; 2]


In [114]:
let is_even x = x mod 2 == 0 in
List.filter is_even [1; 2; 3; 4; 5; 6; 7; 8; 9; 10]

- : int list = [2; 4; 6; 8; 10]


## Flattening

Sometimes we need to map one object to multiple other cases.  One way to do this is to map each element to a list.  

In [115]:
List.map (fun x -> [x; x+1; x+2]) [1;2;3]

- : int list list = [[1; 2; 3]; [2; 3; 4]; [3; 4; 5]]


But how do we turn an `int list list` into an `int list`?  `flatten [[e11; e12]; ...; [en1; en2]]` returns `[e11; e12; ...; en1; en2]`.

In [116]:
let x = List.map (fun x -> [x; x+1; x+2]) [1;2;3] in
List.flatten x

- : int list = [1; 2; 3; 2; 3; 4; 3; 4; 5]


### Folding

But my favorite functional list operation is called _folding_.  Specifically, `List.fold_left f init [e0; e1; ...; en]` is defined to be `f (f (f (f init e0) e1) ...) en`.

Yes, this is confusing.  Let's walk through it.  The innermost expression, `f init e0` is computed first.  Intuitively, `f` is combining `init` with `e0`, the first element of the list.  We then take this value and pass it to `f` along with `e1`.

Another way to think about folding is that it _accumulates_ information.  The first argument of `f` is sometimes called the accumulator.  We can accumulate all kinds of information.  We can use it to count how many list elements we've seen.  We can use it to add all the numbers of the list.  It's very powerful.

In [117]:
List.fold_left

- : ('a -> 'b -> 'a) -> 'a -> 'b list -> 'a = <fun>


Here `'a` is the type of the accumulator, and `'b` is the type of list elements.  So `f` combines an accumulator value (`'a`) and a list element (`'b`).  The second argument is the initial accumulator value.  The third argument is the list.  Note that fold returns the final accumulator value.

Ok, let's look at an example.  Let's compute the length of a list.

Ok, let's try to write a function that computes the length of a list.

In [118]:
let list_length l = List.fold_left (fun acc element -> acc + 1) 0 l

val list_length : 'a list -> int = <fun>


There's a lot going on here!  We're going to _accumulate_ the number of list elements we've seen so far.  Our function takes the old accumulator and the next list element.  But since we're just counting the number of elements, we actually don't need to look at the element at all.  So we just take the old accumulator and add one.

We also set `0` as the initial accumulator.  That's because we're counting, and before we have seen any list elements, the count is 0.

In [119]:
list_length [1; 2; 3]

- : int = 3


In [120]:
list_length []

- : int = 0


Let's try something a little more interesting.  Let's compute the sum of all integers in a list.

In [121]:
let sumlist l = List.fold_left (fun accum element -> accum + element) 0 l

val sumlist : int list -> int = <fun>


- Accumulator stores the sum of all elements visited so far
- Function `f` takes the old value of the accumulator and simply adds the value of the new element
- We start adding from 0

In [122]:
sumlist [1; 2; 3]

- : int = 6


In [123]:
sumlist []

- : int = 0


Ok, let's do one more trick.  Let's use `fold` to reverse a list.  Our accumulator will actually hold the reversed list.  We'll start with an empty list.  Each time we visit a list element, we'll put it on the front of the accumulated list.

`a :: b` means to put `a` at the front of list `b`.

In [124]:
let reverselist l = List.fold_left (fun accum element -> element :: accum) [] l

val reverselist : 'a list -> 'a list = <fun>


In [125]:
reverselist [1; 2; 3]

- : int list = [3; 2; 1]


## A Fun Example

From [Advent of Code](https://adventofcode.com/2020/day/1)

Specifically, they need you to find the two entries that sum to 2020 and then multiply those two numbers together.
 
For example, suppose your expense report contained the following:
 
 
    1721
    979
    366
    299
    675
    1456

In this list, the two entries that sum to 2020 are 1721 and 299. Multiplying them together produces 1721 * 299 = 514579, so the correct answer is 514579.

So how can we solve this functionally?

1. Read the numbers into a list of strings `["1";"2";"3"]`
2. Convert to a list of integers with `map` `[1;2;3]`
3. Turn the list into a list of all tuples using `map` and `flatten` `[(1,1);(1,2);(1,3);(2,1);(2,2);(2,3);(3,1);(3,2);(3,3)]`
4. Find a tuple that sums to 2020 using `find`: `(n1,n2)`
5. Multiply it!

In [131]:
let input_string = {foo|1865
1179
1328
1390
322
1999
1713
1808
1380
1727
1702
1304
1481
1334
1728
1953
1413
1753
1723
1379
1532
1918
1490
71
1388
1519
807
1427
1729
1952
970
1405
1500
1782
1899
1501
1720
1832
1706
1658
1333
486
1670
1674
1290
1893
1382
1761
1945
1607
1319
1508
1464
1733
1917
1483
1620
1677
1835
1810
1385
1840
1705
1994
1695
1599
1681
1566
1153
1672
1373
1679
1714
1283
1950
1197
1696
1936
1218
1910
1786
958
1565
1583
1914
1853
1682
1267
1543
1322
1965
1406
860
1754
1867
1393
1404
1191
1861
2007
1613
1938
1340
1227
1628
1826
1638
1970
1993
1748
496
1661
1736
1593
1298
1882
1763
1616
1848
92
1338
1707
1587
1996
1708
700
1460
1673
1450
1815
1537
1825
1683
1743
1949
1933
1289
1905
1307
1845
1855
1955
1554
1570
1931
1780
1929
1980
1978
1998
1371
1981
1817
1722
1545
1561
1352
1935
1553
1796
1847
1640
1414
1198
1669
1909
1879
1744
1783
1367
1632
1990
1937
1919
1431
2002
1603
1948
1317
1282
1349
1839
1575
1730
1849
1959
1971
2009
1641
1402
1924
1757
1605
1693
1762
283
1525
1964
1715
1602|foo}

val input_string : string =
  "1865\n1179\n1328\n1390\n322\n1999\n1713\n1808\n1380\n1727\n1702\n1304\n1481\n1334\n1728\n1953\n1413\n1753\n1723\n1379\n1532\n1918\n1490\n71\n1388\n1519\n807\n1427\n1729\n1952\n970\n1405\n1500\n1782\n1899\n1501\n1720\n1832\n1706\n1658\n1333\n486\n1670\n1674\n1290\n1893\n1382\n1761\n1945\n1607\n1319\n1508\n1464\n1733\n1917\n1483\n1620\n1677\n1835\n1810\n1385\n"... (* string length 986; truncated *)


### Read the numbers into a list of strings


In [132]:
let l = Str.split (Str.regexp "\n") input_string

val l : string list =
  ["1865"; "1179"; "1328"; "1390"; "322"; "1999"; "1713"; "1808"; "1380";
   "1727"; "1702"; "1304"; "1481"; "1334"; "1728"; "1953"; "1413"; "1753";
   "1723"; "1379"; "1532"; "1918"; "1490"; "71"; "1388"; "1519"; "807";
   "1427"; "1729"; "1952"; "970"; "1405"; "1500"; "1782"; "1899"; "1501";
   "1720"; "1832"; "1706"; "1658"; "1333"; "486"; "1670"; "1674"; "1290";
   "1893"; "1382"; "1761"; "1945"; "1607"; "1319"; "1508"; "1464"; "1733";
   "1917"; "1483"; "1620"; "1677"; "1835"; "1810"; "1385"; "1840"; "1705";
   "1994"; "1695"; "1599"; "1681"; "1566"; "1153"; "1672"; "1373"; "1679";
   "1714"; "1283"; "1950"; "1197"; "1696"; "1936"; "1218"; "1910"; "1786";
   "958"; "1565"; "1583"; "1914"; "1853"; "1682"; "1267"; "1543"; "1322";
   "1965"; "1406"; "860"; "1754"; "1867"; "1393"; "1404"; "1191"; "1861";
   "2007"; "1613"; "1938"; "1340"; "1227"; "1628"; "1826"; "1638"; "1970";
   "1993"; "1748"; "496"; "1661"; "1736"; "1593"; "1298"; "1882"; "1763";
   "1616"; "

### Convert to a list of integers (`map`)


In [133]:
let l2 = List.map int_of_string l

val l2 : int list =
  [1865; 1179; 1328; 1390; 322; 1999; 1713; 1808; 1380; 1727; 1702; 1304;
   1481; 1334; 1728; 1953; 1413; 1753; 1723; 1379; 1532; 1918; 1490; 71;
   1388; 1519; 807; 1427; 1729; 1952; 970; 1405; 1500; 1782; 1899; 1501;
   1720; 1832; 1706; 1658; 1333; 486; 1670; 1674; 1290; 1893; 1382; 1761;
   1945; 1607; 1319; 1508; 1464; 1733; 1917; 1483; 1620; 1677; 1835; 1810;
   1385; 1840; 1705; 1994; 1695; 1599; 1681; 1566; 1153; 1672; 1373; 1679;
   1714; 1283; 1950; 1197; 1696; 1936; 1218; 1910; 1786; 958; 1565; 1583;
   1914; 1853; 1682; 1267; 1543; 1322; 1965; 1406; 860; 1754; 1867; 1393;
   1404; 1191; 1861; 2007; 1613; 1938; 1340; 1227; 1628; 1826; 1638; 1970;
   1993; 1748; 496; 1661; 1736; 1593; 1298; 1882; 1763; 1616; 1848; 92; 1338;
   1707; 1587; 1996; 1708; 700; 1460; 1673; 1450; 1815; 1537; 1825; 1683;
   1743; 1949; 1933; 1289; 1905; 1307; 1845; 1855; 1955; 1554; 1570; 1931;
   1780; 1929; 1980; 1978; 1998; 1371; 1981; 1817; 1722; 1545; 1561; 1352;
   1935; 15

### Turn the list into a list of all tuples (`map`)

This is the only tricky part.  We'll first write a function `make_tuples n1` that takes a number `n1` and makes a tuple for each input where the first tuple is `n1` and the the second is the input.

In [134]:
let make_tuples n1 = List.map (fun n2 -> (n1, n2)) l2;;

val make_tuples : 'a -> ('a * int) list = <fun>


And now we map over `l` using our new function!

In [135]:
let l3 = List.map make_tuples l2

val l3 : (int * int) list list =
  [[(1865, 1865); (1865, 1179); (1865, 1328); (1865, 1390); (1865, 322);
    (1865, 1999); (1865, 1713); (1865, 1808); (1865, 1380); (1865, 1727);
    (1865, 1702); (1865, 1304); (1865, 1481); (1865, 1334); (1865, 1728);
    (1865, 1953); (1865, 1413); (1865, 1753); (1865, 1723); (1865, 1379);
    (1865, 1532); (1865, 1918); (1865, 1490); (1865, 71); (1865, 1388);
    (1865, 1519); (1865, 807); (1865, 1427); (1865, 1729); (1865, 1952);
    (1865, 970); (1865, 1405); (1865, 1500); (1865, 1782); (1865, 1899);
    (1865, 1501); (1865, 1720); (1865, 1832); (1865, 1706); (1865, 1658);
    (1865, 1333); (1865, 486); (1865, 1670); (1865, 1674); (1865, 1290);
    (1865, 1893); (1865, 1382); (1865, 1761); (1865, 1945); (1865, 1607);
    (1865, 1319); (1865, 1508); (1865, 1464); (1865, 1733); (1865, 1917);
    (1865, 1483); (1865, 1620); (1865, 1677); (1865, 1835); (1865, 1810);
    (1865, 1385); (1865, 1840); (1865, 1705); (1865, 1994); (1865, 1695);
    (1865, 

There is a small problem.  We have a `string list list` and want a `string list`.  `flatten` to the rescue!

In [136]:
let l4 = List.flatten l3

val l4 : (int * int) list =
  [(1865, 1865); (1865, 1179); (1865, 1328); (1865, 1390); (1865, 322);
   (1865, 1999); (1865, 1713); (1865, 1808); (1865, 1380); (1865, 1727);
   (1865, 1702); (1865, 1304); (1865, 1481); (1865, 1334); (1865, 1728);
   (1865, 1953); (1865, 1413); (1865, 1753); (1865, 1723); (1865, 1379);
   (1865, 1532); (1865, 1918); (1865, 1490); (1865, 71); (1865, 1388);
   (1865, 1519); (1865, 807); (1865, 1427); (1865, 1729); (1865, 1952);
   (1865, 970); (1865, 1405); (1865, 1500); (1865, 1782); (1865, 1899);
   (1865, 1501); (1865, 1720); (1865, 1832); (1865, 1706); (1865, 1658);
   (1865, 1333); (1865, 486); (1865, 1670); (1865, 1674); (1865, 1290);
   (1865, 1893); (1865, 1382); (1865, 1761); (1865, 1945); (1865, 1607);
   (1865, 1319); (1865, 1508); (1865, 1464); (1865, 1733); (1865, 1917);
   (1865, 1483); (1865, 1620); (1865, 1677); (1865, 1835); (1865, 1810);
   (1865, 1385); (1865, 1840); (1865, 1705); (1865, 1994); (1865, 1695);
   (1865, 1599); (1865, 1681)

### Find a tuple that sums to 2020

Instead of `List.filter` we'll use `List.find` here.  Instead of returning a list, `find f l` returns the first element in `l` that `f` returns `true` on.

In [137]:
let tuple_check (a,b) = a+b == 2020;;
let special_tuple = List.find tuple_check l4;;

val tuple_check : int * int -> bool = <fun>


val special_tuple : int * int = (71, 1949)


In [140]:
(fst special_tuple) * (snd special_tuple)

- : int = 138379


## Variables

There are two types in variables in OCaml:
- Immutable
- Mutable

Functional programming is based on immutable variables, so we'll explore the let binding, which allows us to bind an expression to a variable.


In [126]:
"Hello" ^ " World!"

- : string = "Hello World!"


In [127]:
let s1 = "Hello" in
let s2 = " world!" in
s1 ^ s2

- : string = "Hello world!"


In [128]:
let discount = 0.1 in
let remain = 1.0 -. discount in
let msrp = 50000.0 in
msrp *. remain

- : float = 45000.


Variables in procedural languages are generally _mutable_, which means that they can hold different values at different times.  In OCaml, this is accomplished with a `ref` (reference) type.  As with `list` types, `ref` types take an argument that comes first, and describes the type of the object being stored.  For example, `int ref` is the type of a mutable integer, and `int list ref` is the type of a mutable list of integers.

`ref e` creates a reference initially to `e`.  To give a name to a reference variable, we usually use a let.  To access the current value of `v` we use `!v`.

In [129]:
let v = ref 42 in
!v

- : int = 42


In [130]:
let v = ref 42 in
v := 43;
!v

- : int = 43
